In [1]:
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import torch.utils.data as data
import torchvision
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


In [2]:
transform = transforms.Compose([
transforms.ToTensor(),
transforms.Grayscale(num_output_channels=1),
transforms.Normalize(mean=[0.5,], std=[0.5,]),
])

In [3]:
data = torchvision.datasets.ImageFolder(root='./archive/data/extracted_images/', transform=transform)

In [4]:
trainsize = int(len(data)*0.75)
testsize = len(data)-trainsize

In [5]:
train_set, test_set = torch.utils.data.random_split(data, (trainsize,testsize))

In [6]:
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True,  num_workers=1)
test_data_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True,  num_workers=1)

In [7]:
class Net(nn.Module):   
  def __init__(self):
      super(Net, self).__init__()

      self.cnn_layers = nn.Sequential(
          # Defining a 2D convolution layer
          nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(4),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          # Defining another 2D convolution layer
          nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(4),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
      )

      self.linear_layers = nn.Sequential(
          nn.Linear(484, 82)
      )

  # Defining the forward pass    
  def forward(self, x):
      x = self.cnn_layers(x)
      x = x.view(x.size(0), -1)
      x = self.linear_layers(x)
      return x

In [8]:
model = Net()
# defining the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=484, out_features=82, bias=True)
  )
)


In [10]:
for i in range(10):
    running_loss = 0
    correct = 0
    for images, labels in train_data_loader:

        if torch.cuda.is_available():
          images = images.cuda()
          labels = labels.cuda()

        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
        correct += (predicted == labels).sum().item()
    accuracy = 100 * (correct / (len(train_data_loader)*32))
    print(accuracy)
    print("Epoch {} - Training loss: {}".format(i+1, running_loss/len(train_data_loader)))

C:\Users\VivekKuleen\anaconda3\envs\reg\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


79.28676804357694
Epoch 1 - Training loss: 0.7981486507537984
85.92047775760327
Epoch 2 - Training loss: 0.5117747645560505
88.21812585111212
Epoch 3 - Training loss: 0.41919635673473354
89.12739729913754
Epoch 4 - Training loss: 0.38262471149616784
89.6859396277803
Epoch 5 - Training loss: 0.3624864388671562
90.0015603722197
Epoch 6 - Training loss: 0.3476395557230382
90.39200805719474
Epoch 7 - Training loss: 0.3354759863493233
90.53315081706764
Epoch 8 - Training loss: 0.32908969350517236
90.69308896958694
Epoch 9 - Training loss: 0.3211075116116537
90.85444564230595
Epoch 10 - Training loss: 0.31619349061044316


In [11]:
correct_count, all_count = 0, 0
for images,labels in test_data_loader:
  for i in range(len(labels)):
    if torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    img = images[i].view(1, 1, 45, 45)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.cpu()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.cpu()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 93994

Model Accuracy = 0.8275528225205864
